#  Setup stuff: Connectivity

In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


In [ ]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


#  Finally now, the Analytics ..

In [ ]:

#  What's in the graph ..

l_result1 = my_graph.query("""
   MATCH ( n ) 
   RETURN n
   """)
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query("""
   MATCH ( n ) - [ r ] -> ( m )
   RETURN r
   """)
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


In [ ]:

#  What's in the edge .. 

l_result = my_graph.query("""

   MATCH ( n )  - [r] ->  (m)
   RETURN ID(n) AS FROM, ID(m) AS TO, r.DISTANCE AS DISTANCE, r.NUM_HOPS AS NUM_HOPS, TYPE(r) AS my_type, LABELS(n) AS FROM_LABELS, LABELS(m) AS TO_LABELS
   
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))


In [ ]:

l_result = my_graph.query("""

   MATCH (n: Airport) - [ r ] -> (m: Airport)
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


In [ ]:
#################################################################

In [ ]:

#  https://stackoverflow.com/questions/27600608/traverse-path-cypher-query
#
#  Just direct flight between SJC and MKE   (none, there is no direct flight)

l_result = my_graph.query("""

   MATCH (n: Airport ) - [ r ] -> (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


In [ ]:

#  First traversal, Airport --> Airport, up to 3 in distance ..

l_result = my_graph.query("""

   MATCH (n: Airport ) - [r *..3] -> (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN n.airport_code, m.airport_code, "3" AS my_length
   
   """)

print(l_result[0:30])

#  Sample output,
#
#       n.airport_code m.airport_code my_length
#     0            SJC            MKE         3
#     1            SJC            MKE         3


In [ ]:

#  Adding path ..

l_result = my_graph.query("""

   MATCH p= ( n: Airport ) - [r *..3] -> (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN n.airport_code AS FROM, m.airport_code AS TO, p
   """)

print(l_result[0:30])

#  Currently, error ..
#
#  OperationError: query 9tWneccAey6jzpKLGGtto7v3X1iSkQDdRrSa7gcuv8fD-3qJysJrdkEo1NWz7V error: backtrace
#  (QueryClient.cpp:456): opgraph check failed (OpGraph.cpp:429): Unimplemented Features Pass
#  (OpGraphErrorChecker.cpp:12): backtrace (UnimplementedFeaturesChecker.cpp:360): backtrace
#  (OpGraph.cpp:682): backtrace (OpGraph.cpp:682): backtrace (OpGraph.cpp:682): backtrace
#  (OpGraph.cpp:682): backtrace (OpGraph.cpp:658): backtrace (UnimplementedFeaturesChecker.cpp:214):
#  accessing path variable p is not supported (UnimplementedFeaturesChecker.cpp:24): InternalError:NotImplemented


In [13]:

#  Using COLLECT to try and get path ..

l_result = my_graph.query("""

   MATCH ( n: Airport ) - [r *..3] -> (m: Airport)
   WHERE n.id = "SJC" AND m.id = "MKE"
   
   RETURN n.id AS FROM, COLLECT(m.id)
   // RETURN COLLECT( [n.airport_code, m.airport_code] )                //  This did nothing effective
   
   """)

print(l_result[0:30])

#  This did not used to error. It used to return,
#
#    COLLECT(m.airport_code) FROM
#  0              [MKE, MKE]  SJC
# 
#  Not what we want

#  Now this gives,
#
#  OperationError: 9HaUVQpn6SQh83sR3oDGZLmtyzUoBgxEZeU26316cb4B-3Hc5zx68K6VVZyswB backtrace
#  (QueryClient.cpp:575): opgraph check failed (OpGraph.cpp:564): Unimplemented Features Pass 
#  (OpGraphErrorChecker.cpp:12): backtrace (UnimplementedFeaturesChecker.cpp:376): backtrace 
#  (OpGraph.cpp:822): backtrace (OpGraph.cpp:822): backtrace (OpGraph.cpp:798):
#  returning named variable paths not yet supported (UnimplementedFeaturesChecker.cpp:197):
#  InternalError:NotImplemented: InternalError:NotImplemented


          0/? [?op/s]

OperationError: 57TcsTkWDpoAXW5Lxw4Q2NrKqeVQv1Tg9iy9dLgipakc-585Zo8PozgQviNPv7 backtrace (QueryOperation.cpp:67): backtrace (Network.h:331): backtrace (QueryOperation.cpp:70): backtrace (QueryClient.cpp:571): opgraph check failed (OpGraph.cpp:458): Unimplemented Features Pass (OpGraphErrorChecker.cpp:12): backtrace (UnimplementedFeaturesChecker.cpp:386): backtrace (OpGraph.cpp:716): backtrace (OpGraph.cpp:716): backtrace (OpGraph.cpp:692): returning named variable paths not yet supported (UnimplementedFeaturesChecker.cpp:186): InternalError:NotImplemented: InternalError:NotImplemented: InternalError:NotImplemented

In [ ]:
#  Another attempt at using COLLECT, on the edge this time ..

l_result = my_graph.query("""

   MATCH ( n: Airport )  - [r *..3] ->  (m: Airport)
   WHERE n.id = "SJC" AND m.id = "MKE"
   RETURN n.id AS FROM, COLLECT(r.END_ID)
   """)

print(l_result[0:30])


#  OperationError: query 6jxiUgnCFao3yf2BTXPJqQDUTkAwTL4LMzmKjF6kNmD4-4BeDpVqisXe9Ha8QW error:
#  backtrace (QueryClient.cpp:452): backtrace (OpGraph.cpp:313): backtrace (OpGraph.h:588):
#  backtrace (OpGraph.h:588): backtrace (OpGraph.h:588): backtrace (OpGraph.h:588): backtrace
#  (OpGraph.cpp:320): backtrace (RewritePattern.h:46): backtrace (MatchOrderOptimizer.cpp:102):
#  backtrace (MatchOperator.cpp:2062): returning named variable paths not yet supported
#  (MatchOperator.cpp:1894): InternalError:NotImplemented


In [ ]:
# ##################################################################

In [ ]:
#  Second traversal, Airport --> Airport, up to 3 in distance, but in other direction ..

l_result = my_graph.query("""

   MATCH (n: Airport )  - [r: FLIES_TO *..3] ->  (m: Airport)
   WHERE n.id = "MKE" AND m.id = "SJC"
   RETURN n.id, m.id
   
   """)

# print(l_result[0:30])
print(tabulate(l_result, headers='keys', tablefmt='psql'))


In [ ]:
# ##################################################################

In [ ]:

#  From Roshan and Boris ..

l_result = my_graph.query("""

   MATCH ( n: Airport )  - [] ->  (m: Airport)
   WHERE n.id = "SJC" AND m.id = "MKE"
   RETURN "1" AS num_hops, n.id AS FROM, [m.id] as path
      
   UNION
   
   MATCH ( n: Airport )-[]->(p1:Airport)-[]-> (m: Airport)
   WHERE n.id = "SJC" AND m.id = "MKE"
   RETURN "2" AS num_hops, n.id AS FROM, [p1.id,m.id] as path
      
   UNION
   
   MATCH ( n: Airport )-[]->(p1:Airport)-[]->(p2:Airport)-[]-> (m: Airport)
   WHERE n.id = "SJC" AND m.id = "MKE"
   RETURN "3" AS num_hops, n.id AS FROM, [p1.id,p2.id,m.id] as path
   
   """)

print(l_result[0:30])
    
#  Outputs,
#
#    FROM num_hops             path
#  0  SJC        3  [DEN, ORD, MKE]
#  1  SJC        2       [ORD, MKE]

